# Why feature learning is better than simple propositionalization

**NOTE: Due to featuretools's and tsfresh's memory requirements, this notebook will not run on MyBinder when RUN_FEATURETOOLS=True RUN_TSFRESH=True.**

In this notebook we will compare getML to featuretools and tsfresh, both of which open-source libraries for feature engineering. We find that advanced algorithms featured in getML yield significantly better predictions on this dataset. We then discuss why that is.

Summary:

- Prediction type: __Regression model__
- Domain: __Air pollution__
- Prediction target: __pm 2.5 concentration__ 
- Source data: __Multivariate time series__
- Population size: __41757__

*Author: Dr. Patrick Urbanke*

## Background

Many data scientists and AutoML tools use propositionalization methods for feature engineering. These propositionalization methods usually work as follows:

- Generate a large number of hard-coded features
- Use feature selection to pick a percentage of these features

By contrast, getML (https://getml.com/product) contains approaches for feature learning: Feature learning adapts machine learning approaches such as decision trees or gradient boosting to the problem of extracting features from relational data and time series.

In this notebook, we will benchmark getML (https://getml.com/product) against featuretools (https://www.featuretools.com/) and tsfresh (https://tsfresh.readthedocs.io/en/latest/). Both of these libaries use propositionalization approaches for feature engineering.

As our example dataset, we use a publicly available dataset on air pollution in Beijing, China (https://archive.ics.uci.edu/ml/datasets/Beijing+PM2.5+Data). The data set has been originally used in the following study:

Liang, X., Zou, T., Guo, B., Li, S., Zhang, H., Zhang, S., Huang, H. and Chen, S. X. (2015). Assessing Beijing's PM2.5 pollution: severity, weather impact, APEC and winter heating. Proceedings of the Royal Society A, 471, 20150257.

We find that getML significantly outperforms featuretools and tsfresh in terms of predictive accuracy (**R-squared of 62.3%** vs **R-squared of 50.4%**).

Our findings indicate that getML's feature learning algorithms are better at adapting to data sets and are also more scalable due to their lower memory requirement.

## A web frontend for getML

The getML monitor is a frontend built to support your work with getML. The getML monitor displays information such as the imported data frames, trained pipelines and allows easy data and feature exploration. You can launch the getML monitor [here](http://localhost:1709).

In [1]:
import os
from urllib import request

import getml
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

import matplotlib.pyplot as plt
%matplotlib inline  

In [2]:
RUN_FEATURETOOLS = True
RUN_TSFRESH = True

if RUN_FEATURETOOLS:
    from utils import FTTimeSeriesBuilder

if RUN_TSFRESH:
    from utils import TSFreshBuilder

## 1. Loading data

### 1.1 Download from source

We begin by downloading the data from the UCI Machine Learning repository.

In [3]:
FEATURETOOLS_FILES = [
    "featuretools_training.csv",
    "featuretools_test.csv"
]

if not RUN_FEATURETOOLS:
    for fname in FEATURETOOLS_FILES:
        if not os.path.exists(fname):
            fname, res = request.urlretrieve(
                "https://static.getml.com/datasets/air_pollution/featuretools/" + fname, 
                fname
            )

In [4]:
TSFRESH_FILES = [
    "tsfresh_training.csv",
    "tsfresh_test.csv"
]

if not RUN_TSFRESH:
    for fname in TSFRESH_FILES:
        if not os.path.exists(fname):
            fname, res = request.urlretrieve(
                "https://static.getml.com/datasets/air_pollution/tsfresh/" + fname, 
                fname
            )

In [5]:
fname = "PRSA_data_2010.1.1-2014.12.31.csv"

if not os.path.exists(fname):
    fname, res = request.urlretrieve(
        "https://archive.ics.uci.edu/ml/machine-learning-databases/00381/" + fname, 
        fname
    )


### 1.2 Prepare data for tsfresh and getML

Our our goal is to predict the pm2.5 concentration from factors such as weather or time of day. However, there are some **missing entries** for pm2.5, so we get rid of them.

In [6]:
data_full_pandas = pd.read_csv(fname)

data_full_pandas = data_full_pandas[
    data_full_pandas["pm2.5"] == data_full_pandas["pm2.5"]
]

tsfresh requires a date column, so we build one.

In [7]:
def add_leading_zero(val):
    if len(str(val)) == 1:
        return "0" + str(val)
    return str(val)

data_full_pandas["month"] = [
    add_leading_zero(val) for val in data_full_pandas["month"]
]

data_full_pandas["day"] = [
    add_leading_zero(val) for val in data_full_pandas["day"]
]

data_full_pandas["hour"] = [
    add_leading_zero(val) for val in data_full_pandas["hour"]
]

def make_date(year, month, day, hour):
    return year + "-" + month + "-" + day + " " + hour + ":00:00"

data_full_pandas["date"] = [
    make_date(str(year), month, day, hour) \
    for year, month, day, hour in zip(
        data_full_pandas["year"],
        data_full_pandas["month"],
        data_full_pandas["day"],
        data_full_pandas["hour"],
    )
]


tsfresh also requires the time series to have ids. Since there is only a single time series, that series has the same id.

In [8]:
data_full_pandas["id"] = 1

The dataset now contains many columns that we do not need or that tsfresh cannot process. For instance, *cbwd* might actually contain useful information, but it is a categorical variable, which is difficult to handle for tsfresh, so we remove it.

We also want to split our data into a training and testing set.

In [9]:
data_train_pandas = data_full_pandas[data_full_pandas["year"] < 2014]
data_test_pandas = data_full_pandas[data_full_pandas["year"] == 2014]
data_full_pandas = data_full_pandas

In [10]:
def remove_unwanted_columns(df):
    del df["cbwd"]
    del df["year"]
    del df["month"]
    del df["day"]
    del df["hour"]
    del df["No"]
    return df

data_full_pandas = remove_unwanted_columns(data_full_pandas)
data_train_pandas = remove_unwanted_columns(data_train_pandas)
data_test_pandas = remove_unwanted_columns(data_test_pandas)

In [11]:
data_full_pandas

,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,date,id
24,129.0,-16,-4.0,1020.0,1.79,0,0,2010-01-02 00:00:00,1
25,148.0,-15,-4.0,1020.0,2.68,0,0,2010-01-02 01:00:00,1
26,159.0,-11,-5.0,1021.0,3.57,0,0,2010-01-02 02:00:00,1
27,181.0,-7,-5.0,1022.0,5.36,1,0,2010-01-02 03:00:00,1
28,138.0,-7,-5.0,1022.0,6.25,2,0,2010-01-02 04:00:00,1
...,...,...,...,...,...,...,...,...,...
43819,8.0,-23,-2.0,1034.0,231.97,0,0,2014-12-31 19:00:00,1
43820,10.0,-22,-3.0,1034.0,237.78,0,0,2014-12-31 20:00:00,1
43821,10.0,-22,-3.0,1034.0,242.70,0,0,2014-12-31 21:00:00,1
43822,8.0,-22,-4.0,1034.0,246.72,0,0,2014-12-31 22:00:00,1


We then **load the data into the getML engine**. We begin by setting a project.

In [12]:
getml.engine.set_project('air_pollution')


Connected to project 'air_pollution'


In [13]:
df_full = getml.data.DataFrame.from_pandas(data_full_pandas, name='full')
df_train = getml.data.DataFrame.from_pandas(data_train_pandas, name='train')
df_test = getml.data.DataFrame.from_pandas(data_test_pandas, name='test')

df_full["date"] = df_full["date"].as_ts()

We need to **assign roles** to the columns, such as defining the target column.

In [14]:
def set_roles(df):
    df.set_role(["date"], getml.data.roles.time_stamp)
    df.set_role(["pm2.5"], getml.data.roles.target)
    df.set_role([
        "DEWP", 
        "TEMP",
        "PRES",
        "Iws",
        "Is",
        "Ir"], getml.data.roles.numerical)

set_roles(df_full)
set_roles(df_train)
set_roles(df_test)

## 3. Predictive modelling


### 3.1 Pipeline 1: Complex features, 7 days

For our first experiment, we will learn complex features and allow a memory of up to seven days. That means at every given point in time, the algorithm is allowed to back seven days into the past.

getML uses relational learning to build construct the pipelines. Even though there is a simpler time series API, the relational API is more flexible which is why decide to use it.

In [15]:
population = getml.data.Placeholder('population')

peripheral = getml.data.Placeholder('peripheral')

population.join(
    peripheral,
    time_stamp='date',
    memory=getml.data.time.days(7)
)

population

placeholder   other placeholder   allow lagged targets   horizon   join keys used   memory     other join keys used   other time stamps used   relationship   time stamps used   upper time stamps used
  0   population    peripheral          False                  0.0                        604800.0                          date                     many-to-many   date

In [16]:
relmt = getml.feature_learning.RelMTModel(
    num_features=10,
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    seed=4367,
    num_threads=1
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe = getml.pipeline.Pipeline(
    tags=['memory: 7d', 'complex features'],
    population=population,
    peripheral=[peripheral],
    feature_learners=[relmt],
    predictors=[predictor]
)

pipe

Pipeline(preprocessors=[],
         feature_learners=['RelMTModel'],
         feature_selectors=[],
         include_categorical=False,
         peripheral=['peripheral'],
         population='population',
         predictors=['XGBoostRegressor'],
         tags=['memory: 7d', 'complex features'],
         share_selected_features=0.5)

It is good practice to always check your data model first, even though `check(...)` is also called by `fit(...)`. That enables us to make last-minute changes.

In [17]:
pipe.check(df_train, [df_full])

Checking data model...
OK.


We now fit our data on the training set and evaluate our findings, both in-sample and out-of-sample.

In [18]:
pipe.fit(df_train, [df_full])

Checking data model...
OK.

RelMT: Training features...
[========================================] 100%

RelMT: Building features...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:5m:59.089638



Pipeline(preprocessors=[],
         feature_learners=['RelMTModel'],
         feature_selectors=[],
         include_categorical=False,
         peripheral=['peripheral'],
         population='population',
         predictors=['XGBoostRegressor'],
         tags=['memory: 7d', 'complex features'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/air_pollution/8WcKqj/0/

In [19]:
pipe.score(df_test, [df_full])


RelMT: Building features...
[========================================] 100%



,date time,set used,target,mae,rmse,rsquared
0,2021-05-15 13:47:40,train,pm2.5,35.2722,51.1125,0.6901
1,2021-05-15 13:47:58,test,pm2.5,40.0586,57.7904,0.6226


### 3.2 Pipeline 2: Complex features, 1 day

In [20]:
population = getml.data.Placeholder('population')

peripheral = getml.data.Placeholder('peripheral')

population.join(
    peripheral,
    time_stamp='date',
    memory=getml.data.time.days(1)
)

population

placeholder   other placeholder   allow lagged targets   horizon   join keys used   memory    other join keys used   other time stamps used   relationship   time stamps used   upper time stamps used
  0   population    peripheral          False                  0.0                        86400.0                          date                     many-to-many   date

In [21]:
relmt = getml.feature_learning.RelMTModel(
    num_features=10,
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    seed=4367,
    num_threads=1
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe = getml.pipeline.Pipeline(
    tags=['memory: 1d', 'complex features'],
    population=population,
    peripheral=[peripheral],
    feature_learners=[relmt],
    predictors=[predictor]
)

pipe

Pipeline(preprocessors=[],
         feature_learners=['RelMTModel'],
         feature_selectors=[],
         include_categorical=False,
         peripheral=['peripheral'],
         population='population',
         predictors=['XGBoostRegressor'],
         tags=['memory: 1d', 'complex features'],
         share_selected_features=0.5)

In [22]:
pipe.check(df_train, [df_full])

Checking data model...
OK.


In [23]:
pipe.fit(df_train, [df_full])

Checking data model...
OK.

RelMT: Training features...
[========================================] 100%

RelMT: Building features...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:2m:26.176567



Pipeline(preprocessors=[],
         feature_learners=['RelMTModel'],
         feature_selectors=[],
         include_categorical=False,
         peripheral=['peripheral'],
         population='population',
         predictors=['XGBoostRegressor'],
         tags=['memory: 1d', 'complex features'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/air_pollution/IWChcR/0/

In [24]:
pipe.score(df_test, [df_full])


RelMT: Building features...
[========================================] 100%



,date time,set used,target,mae,rmse,rsquared
0,2021-05-15 13:50:34,train,pm2.5,38.7518,56.176,0.6255
1,2021-05-15 13:50:45,test,pm2.5,44.403,66.1336,0.5153


### 3.3 Pipeline 3: Simple features, 7 days

For our third experiment, we will learn simple features and allow a memory of up to seven days.

In [25]:
population = getml.data.Placeholder('population')

peripheral = getml.data.Placeholder('peripheral')

population.join(
    peripheral,
    time_stamp='date',
    memory=getml.data.time.days(7)
)

population

placeholder   other placeholder   allow lagged targets   horizon   join keys used   memory     other join keys used   other time stamps used   relationship   time stamps used   upper time stamps used
  0   population    peripheral          False                  0.0                        604800.0                          date                     many-to-many   date

In [26]:
fast_prop = getml.feature_learning.FastPropModel(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
    aggregation=getml.feature_learning.FastPropModel.agg_sets.All,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe = getml.pipeline.Pipeline(
    tags=['memory: 7d', 'simple features'],
    population=population,
    peripheral=[peripheral],
    feature_learners=[fast_prop],
    predictors=[predictor]
)

pipe

Pipeline(preprocessors=[],
         feature_learners=['FastPropModel'],
         feature_selectors=[],
         include_categorical=False,
         peripheral=['peripheral'],
         population='population',
         predictors=['XGBoostRegressor'],
         tags=['memory: 7d', 'simple features'],
         share_selected_features=0.5)

In [27]:
pipe.check(df_train, [df_full])

Checking data model...
OK.


In [28]:
pipe.fit(df_train, [df_full])

Checking data model...
OK.

FastProp: Trying 226 features...
[========================================] 100%

FastProp: Building features...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:2m:21.02471



Pipeline(preprocessors=[],
         feature_learners=['FastPropModel'],
         feature_selectors=[],
         include_categorical=False,
         peripheral=['peripheral'],
         population='population',
         predictors=['XGBoostRegressor'],
         tags=['memory: 7d', 'simple features'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/air_pollution/8nqufU/0/

In [29]:
pipe.score(df_test, [df_full])


FastProp: Building features...
[========================================] 100%



,date time,set used,target,mae,rmse,rsquared
0,2021-05-15 13:53:17,train,pm2.5,37.8727,52.7455,0.6839
1,2021-05-15 13:53:30,test,pm2.5,47.5306,65.6835,0.5203


### 3.4 Pipeline 4: Simple features, 1 day

For our fourth experiment, we will learn simple features and allow a memory of up to one day.

In [30]:
population = getml.data.Placeholder('population')

peripheral = getml.data.Placeholder('peripheral')

population.join(
    peripheral,
    time_stamp='date',
    memory=getml.data.time.days(1)
)

population

placeholder   other placeholder   allow lagged targets   horizon   join keys used   memory    other join keys used   other time stamps used   relationship   time stamps used   upper time stamps used
  0   population    peripheral          False                  0.0                        86400.0                          date                     many-to-many   date

In [31]:
fast_prop = getml.feature_learning.FastPropModel(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
    aggregation=getml.feature_learning.FastPropModel.agg_sets.All,
)

predictor = getml.predictors.XGBoostRegressor(n_jobs=1)

pipe = getml.pipeline.Pipeline(
    tags=['memory: 1d', 'simple features'],
    population=population,
    peripheral=[peripheral],
    feature_learners=[fast_prop],
    predictors=[predictor]
)

pipe

Pipeline(preprocessors=[],
         feature_learners=['FastPropModel'],
         feature_selectors=[],
         include_categorical=False,
         peripheral=['peripheral'],
         population='population',
         predictors=['XGBoostRegressor'],
         tags=['memory: 1d', 'simple features'],
         share_selected_features=0.5)

In [32]:
pipe.check(df_train, [df_full])

Checking data model...
OK.


In [33]:
pipe.fit(df_train, [df_full])

Checking data model...
OK.

FastProp: Trying 226 features...
[========================================] 100%

FastProp: Building features...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:58.258886



Pipeline(preprocessors=[],
         feature_learners=['FastPropModel'],
         feature_selectors=[],
         include_categorical=False,
         peripheral=['peripheral'],
         population='population',
         predictors=['XGBoostRegressor'],
         tags=['memory: 1d', 'simple features'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/air_pollution/XhVHdB/0/

In [34]:
pipe.score(df_test, [df_full])


FastProp: Building features...
[========================================] 100%



,date time,set used,target,mae,rmse,rsquared
0,2021-05-15 13:54:39,train,pm2.5,38.5582,55.3213,0.644
1,2021-05-15 13:54:43,test,pm2.5,44.5867,63.5967,0.546


### 3.5 Using featuretools

To make things a bit easier, we have written a high-level wrapper around featuretools which we placed in a separate module (`utils`).

In [35]:
if RUN_FEATURETOOLS:
    ft_builder = FTTimeSeriesBuilder(
        num_features=200,
        horizon=pd.Timedelta(days=0),
        memory=pd.Timedelta(days=1),
        column_id="id",
        time_stamp="date",
        target="pm2.5")
    #
    featuretools_training = ft_builder.fit(data_train_pandas)
    featuretools_test = ft_builder.transform(data_test_pandas)
    #
    featuretools_training.to_csv("featuretools_training.csv", index=False)
    featuretools_test.to_csv("featuretools_test.csv", index=False)

featuretools: Trying features...


/home/patrick/.local/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'entropy', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


Selecting the best out of 133 features...
Time taken: 0h:16m:5.67805



/home/patrick/.local/lib/python3.9/site-packages/featuretools/synthesis/dfs.py:317: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'entropy', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible variable types for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [36]:
if not RUN_FEATURETOOLS:
    featuretools_training = pd.read_csv("featuretools_training.csv")
    featuretools_test = pd.read_csv("featuretools_test.csv")

In [37]:
df_featuretools_training = getml.data.DataFrame.from_pandas(featuretools_training, name='featuretools_training')
df_featuretools_test = getml.data.DataFrame.from_pandas(featuretools_test, name='featuretools_test')

In [38]:
def set_roles_featuretools(df):
    df["date"] = df["date"].as_ts()
    df.set_role(["pm2.5"], getml.data.roles.target)
    df.set_role(["date"], getml.data.roles.time_stamp)
    df.set_role(df.unused_names, getml.data.roles.numerical)
    df.set_role(["id"], getml.data.roles.unused_float)
    return df

df_featuretools_training = set_roles_featuretools(df_featuretools_training)
df_featuretools_test = set_roles_featuretools(df_featuretools_test)

In [39]:
predictor = getml.predictors.XGBoostRegressor()

pipe = getml.pipeline.Pipeline(
    tags=['featuretools', 'memory: 1d'],
    predictors=[predictor]
)

pipe

Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['featuretools', 'memory: 1d'],
         share_selected_features=0.5)

In [40]:
pipe.check(df_featuretools_training)

Checking data model...
OK.


In [41]:
pipe.fit(df_featuretools_training)

Checking data model...
OK.

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:13.784925



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['featuretools', 'memory: 1d'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/air_pollution/3hEWN4/0/

In [42]:
pipe.score(df_featuretools_test)

,date time,set used,target,mae,rmse,rsquared
0,2021-05-15 14:15:31,featuretools_training,pm2.5,38.0455,54.4693,0.6567
1,2021-05-15 14:15:31,featuretools_test,pm2.5,45.3084,64.2717,0.5373


### 3.6 Using tsfresh

tsfresh is a rather low-level library. To make things a bit easier, we have written a high-level wrapper which we placed in a separate module (`utils`).

To limit the memory consumption, we undertake the following steps:

- We limit ourselves to a memory of 1 day from any point in time. This is necessary, because tsfresh duplicates records for every time stamp. That means that looking back 7 days instead of one day, the memory consumption would be  seven times as high.
- We extract only tsfresh's **MinimalFCParameters** and **IndexBasedFCParameters** (the latter is a superset of **TimeBasedFCParameters**).

In order to make sure that tsfresh's features can be compared to getML's features, we also do the following:

- We apply tsfresh's built-in feature selection algorithm.
- Of the remaining features, we only keep the 40 features most correlated with the target (in terms of the absolute value of the correlation).
- We add the original columns as additional features.


In [43]:
data_train_pandas

,pm2.5,DEWP,TEMP,PRES,Iws,Is,Ir,date,id
24,129.0,-16,-4.0,1020.0,1.79,0,0,2010-01-02 00:00:00,1
25,148.0,-15,-4.0,1020.0,2.68,0,0,2010-01-02 01:00:00,1
26,159.0,-11,-5.0,1021.0,3.57,0,0,2010-01-02 02:00:00,1
27,181.0,-7,-5.0,1022.0,5.36,1,0,2010-01-02 03:00:00,1
28,138.0,-7,-5.0,1022.0,6.25,2,0,2010-01-02 04:00:00,1
...,...,...,...,...,...,...,...,...,...
35059,22.0,-19,7.0,1013.0,114.87,0,0,2013-12-31 19:00:00,1
35060,18.0,-21,7.0,1014.0,119.79,0,0,2013-12-31 20:00:00,1
35061,23.0,-21,7.0,1014.0,125.60,0,0,2013-12-31 21:00:00,1
35062,20.0,-21,6.0,1014.0,130.52,0,0,2013-12-31 22:00:00,1


One of the issues about tsfresh is that is actually requires more memory than allowed by MyBinder. We therefore have to remove the parts that relate to this.

In [44]:
if RUN_TSFRESH:
    tsfresh_builder = TSFreshBuilder(
        num_features=200,
        memory=24,
        column_id="id",
        time_stamp="date",
        target="pm2.5")
    #
    tsfresh_training = tsfresh_builder.fit(data_train_pandas)
    tsfresh_test = tsfresh_builder.transform(data_test_pandas)
    #
    tsfresh_training.to_csv("tsfresh_training.csv", index=False)
    tsfresh_test.to_csv("tsfresh_test.csv", index=False)

Feature Extraction: 100%|██████████| 20/20 [01:46<00:00,  5.32s/it]


Selecting the best out of 66 features...
Time taken: 0h:4m:36.226761



Feature Extraction: 100%|██████████| 20/20 [00:26<00:00,  1.33s/it]


tsfresh does not contain built-in machine learning algorithms. In order to ensure a fair comparison, we use the exact same machine learning algorithm we have also used for getML: An XGBoost regressor with all hyperparameters set to their default value.

In order to do so, we load the tsfresh features into the getML engine.

In [45]:
if not RUN_TSFRESH:
    tsfresh_training = pd.read_csv("tsfresh_training.csv")
    tsfresh_test = pd.read_csv("tsfresh_test.csv")

In [46]:
df_tsfresh_training = getml.data.DataFrame.from_pandas(tsfresh_training, name='tsfresh_training')
df_tsfresh_test = getml.data.DataFrame.from_pandas(tsfresh_test, name='tsfresh_test')

As usual, we need to set roles:

In [47]:
def set_roles_tsfresh(df):
    df["date"] = df["date"].as_ts()
    df.set_role(["pm2.5"], getml.data.roles.target)
    df.set_role(["date"], getml.data.roles.time_stamp)
    df.set_role(df.unused_names, getml.data.roles.numerical)
    df.set_role(["id"], getml.data.roles.unused_float)
    return df

df_tsfresh_training = set_roles_tsfresh(df_tsfresh_training)
df_tsfresh_test = set_roles_tsfresh(df_tsfresh_test)

In this case, our pipeline is very simple. It only consists of a single XGBoostRegressor.

In [48]:
predictor = getml.predictors.XGBoostRegressor()

pipe = getml.pipeline.Pipeline(
    tags=['tsfresh', 'memory: 1d'],
    predictors=[predictor]
)

pipe

Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['tsfresh', 'memory: 1d'],
         share_selected_features=0.5)

In [49]:
pipe.check(df_tsfresh_training)

Checking data model...
OK.


In [50]:
pipe.fit(df_tsfresh_training)

Checking data model...
OK.

XGBoost: Training as predictor...
[========================================] 100%

Trained pipeline.
Time taken: 0h:0m:8.759686



Pipeline(preprocessors=[],
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         peripheral=[],
         population='POPULATION',
         predictors=['XGBoostRegressor'],
         tags=['tsfresh', 'memory: 1d'],
         share_selected_features=0.5)

url: http://localhost:1709/#/getpipeline/air_pollution/DXuLoh/0/

In [51]:
pipe.score(df_tsfresh_test)

,date time,set used,target,mae,rmse,rsquared
0,2021-05-15 14:21:29,tsfresh_training,pm2.5,40.7577,57.8644,0.6098
1,2021-05-15 14:21:29,tsfresh_test,pm2.5,46.4913,65.9863,0.5087


## 4. Discussion

We have seen that getML outperforms tsfresh by more than 10 percentage points in terms of R-squared. We now want to analyze why that is.

There are two possible hypotheses:

- getML outperforms featuretools and tsfresh, because it using feature learning and is able to produce more complex features
- getML outperforms featuretools and tsfresh, because it makes better use of memory and is able to look back further.

Let's summarize our findings:


Name         | Memory  | Feature complexity | R-squared | RMSE | 
------------ | ------- | ------------------ | --------- | ---- | 
Pipeline 1   |  7 days |            complex |     62.3% | 57.8 |
Pipeline 2   |   1 day |            complex |     51.5% | 66.1 |
Pipeline 3   |  7 days |             simple |     52.0% | 65.7 |
Pipeline 4   |   1 day |             simple |     54.6% | 63.6 |
featuretools |   1 day |             simple |     53.7% | 64.3 |
tsfresh      |   1 day |             simple |     50.9% | 66.0 |


The summary table shows that combination of both of our hypotheses explains why getML outperforms featuretools and tsfresh. Complex features do better than simple features with a memory of one day. With a memory of seven days, simple features actually get worse. But when you look back seven days and allow more complex features, you get good results.

This suggests that getML outperforms featuretools and tsfresh, because it can make more efficient use of memory and thus look back further. Because RelMT uses feature learning and can build more complex features it can make better use of the greater look-back window.

## 5. Conclusion

We have compared getML's feature learning algorithms to tsfresh's brute-force feature engineering approaches on a data set related to air pollution in China. We found that getML significantly outperforms featuretools and tsfresh. These results are consistent with the view that feature learning can yield significant improvements over simple propositionalization approaches.

However, there are other datasets on which simple propositionalization performs well. Our suggestion is therefore to think of algorithms like `FastProp` and `RelMT` as tools in a toolbox. If a simple tool like `FastProp` gets the job done, then use that. But when you need more advanced approaches, like `RelMT`, you should have them at your disposal as well.

You are encouraged to reproduce these results. You will need getML (https://getml.com/product) and tsfresh (https://tsfresh.readthedocs.io/en/latest/). You can download both for free. 